In [1]:
datafile = "titanicdata.htm"

In [2]:
from bs4 import BeautifulSoup
with open(datafile,encoding="ISO-8859-1") as f:
    soup = BeautifulSoup(f, "html.parser")

In [3]:
table = soup.find('table')

In [4]:
import pandas as pd
data = pd.read_html(str(table), flavor='bs4')[0]

In [5]:
data.head()

,Name,Age,Class/Dept,Ticket,Joined,Job,Boat [Body],Unnamed: 7
0,"ABÄ«-AL-MUNÃ , Mr NÄsÄ«f QÄsim",27,3rd Class Passenger,2699£18 15s 9d,Cherbourg,NaN,15,NaN
1,"ABBING, Mr Anthony",42,3rd Class Passenger,5547£7 11s,Southampton,Blacksmith,NaN,NaN
2,"ABBOTT, Mrs Rhoda Mary 'Rosa'",39,3rd Class Passenger,CA2673£20 5s,Southampton,NaN,A,NaN
3,"ABBOTT, Mr Rossmore Edward",16,3rd Class Passenger,CA2673£20 5s,Southampton,Jeweller,[190],NaN
4,"ABBOTT, Mr Eugene Joseph",13,3rd Class Passenger,CA2673£20 5s,Southampton,Scholar,NaN,NaN


In [6]:
def cleanup(value):
    value = str(value)
    return bytes(value,'utf8').decode('ascii',errors='replace').replace("?","")

In [7]:
data['Name'] = data['Name'].apply(cleanup)
#data['Boat [Body]'] = data['Boat [Body]'].apply(pd.to_numeric, errors='coerce')
data['Age'] = data['Age'].apply(pd.to_numeric, errors='coerce')
data = data[['Name','Age','Class/Dept','Boat [Body]']]

In [8]:
data.head()

,Name,Age,Class/Dept,Boat [Body]
0,"AB����-AL-MUN����, Mr N����s����f Q����sim",27.0,3rd Class Passenger,15
1,"ABBING, Mr Anthony",42.0,3rd Class Passenger,NaN
2,"ABBOTT, Mrs Rhoda Mary 'Rosa'",39.0,3rd Class Passenger,A
3,"ABBOTT, Mr Rossmore Edward",16.0,3rd Class Passenger,[190]
4,"ABBOTT, Mr Eugene Joseph",13.0,3rd Class Passenger,NaN


In [9]:
def checkPass(class_type):
    if "Passenger" in class_type:
        return "Passenger"
    else:
        return "Crew"

data["Crew/Pass"] = data["Class/Dept"].apply(checkPass)
data.head()

,Name,Age,Class/Dept,Boat [Body],Crew/Pass
0,"AB����-AL-MUN����, Mr N����s����f Q����sim",27.0,3rd Class Passenger,15,Passenger
1,"ABBING, Mr Anthony",42.0,3rd Class Passenger,NaN,Passenger
2,"ABBOTT, Mrs Rhoda Mary 'Rosa'",39.0,3rd Class Passenger,A,Passenger
3,"ABBOTT, Mr Rossmore Edward",16.0,3rd Class Passenger,[190],Passenger
4,"ABBOTT, Mr Eugene Joseph",13.0,3rd Class Passenger,NaN,Passenger


In [10]:
def checkClass(class_type):
    if "Passenger" in class_type:
        return class_type.split(" ")[0]
    else:
        return "Crew"

data["Class"] = data["Class/Dept"].apply(checkClass)
data.head()

,Name,Age,Class/Dept,Boat [Body],Crew/Pass,Class
0,"AB����-AL-MUN����, Mr N����s����f Q����sim",27.0,3rd Class Passenger,15,Passenger,3rd
1,"ABBING, Mr Anthony",42.0,3rd Class Passenger,NaN,Passenger,3rd
2,"ABBOTT, Mrs Rhoda Mary 'Rosa'",39.0,3rd Class Passenger,A,Passenger,3rd
3,"ABBOTT, Mr Rossmore Edward",16.0,3rd Class Passenger,[190],Passenger,3rd
4,"ABBOTT, Mr Eugene Joseph",13.0,3rd Class Passenger,NaN,Passenger,3rd


In [11]:
def checkAdult(age):
    if age > 18:
        return "Adult"
    else:
        return "Child"
    
data["Adult/Child"] = data["Age"].apply(checkAdult)    
data.head()

,Name,Age,Class/Dept,Boat [Body],Crew/Pass,Class,Adult/Child
0,"AB����-AL-MUN����, Mr N����s����f Q����sim",27.0,3rd Class Passenger,15,Passenger,3rd,Adult
1,"ABBING, Mr Anthony",42.0,3rd Class Passenger,NaN,Passenger,3rd,Adult
2,"ABBOTT, Mrs Rhoda Mary 'Rosa'",39.0,3rd Class Passenger,A,Passenger,3rd,Adult
3,"ABBOTT, Mr Rossmore Edward",16.0,3rd Class Passenger,[190],Passenger,3rd,Child
4,"ABBOTT, Mr Eugene Joseph",13.0,3rd Class Passenger,NaN,Passenger,3rd,Child


In [12]:
def checkGender(name):
    firstname = name[name.index(",")+2:]
    salutation = firstname.split(" ")[0]
    if(salutation in ["Mr","Master"]):
        return "Male"
    else:
        return "Female"

data["Gender"] = data["Name"].apply(checkGender)
data.head()

,Name,Age,Class/Dept,Boat [Body],Crew/Pass,Class,Adult/Child,Gender
0,"AB����-AL-MUN����, Mr N����s����f Q����sim",27.0,3rd Class Passenger,15,Passenger,3rd,Adult,Male
1,"ABBING, Mr Anthony",42.0,3rd Class Passenger,NaN,Passenger,3rd,Adult,Male
2,"ABBOTT, Mrs Rhoda Mary 'Rosa'",39.0,3rd Class Passenger,A,Passenger,3rd,Adult,Female
3,"ABBOTT, Mr Rossmore Edward",16.0,3rd Class Passenger,[190],Passenger,3rd,Child,Male
4,"ABBOTT, Mr Eugene Joseph",13.0,3rd Class Passenger,NaN,Passenger,3rd,Child,Male


In [13]:
def isNaN(x):
    return str(x) == str(1e400*0)

def checkSurvival(boat):
    if isNaN(boat) or str(boat).strip() == "NaN" or "[" in str(boat):
        return 0
    else:
        return 1

data["Survival"]=data["Boat [Body]"].apply(checkSurvival)    
data.head()

,Name,Age,Class/Dept,Boat [Body],Crew/Pass,Class,Adult/Child,Gender,Survival
0,"AB����-AL-MUN����, Mr N����s����f Q����sim",27.0,3rd Class Passenger,15,Passenger,3rd,Adult,Male,1
1,"ABBING, Mr Anthony",42.0,3rd Class Passenger,NaN,Passenger,3rd,Adult,Male,0
2,"ABBOTT, Mrs Rhoda Mary 'Rosa'",39.0,3rd Class Passenger,A,Passenger,3rd,Adult,Female,1
3,"ABBOTT, Mr Rossmore Edward",16.0,3rd Class Passenger,[190],Passenger,3rd,Child,Male,0
4,"ABBOTT, Mr Eugene Joseph",13.0,3rd Class Passenger,NaN,Passenger,3rd,Child,Male,0


In [14]:
data.groupby(['Crew/Pass'])['Survival'].sum()*100/data.groupby(['Crew/Pass'])['Survival'].count()

Crew/Pass
Crew         13.586957
Passenger    35.798817
Name: Survival, dtype: float64

In [119]:
compare('Class',data)

Class
1st     57.428571
2nd     37.883959
3rd     24.259520
Crew    13.586957
Name: Survival, dtype: float64

In [120]:
compare('Gender',data)

Gender
Female    59.272097
Male      15.540181
Name: Survival, dtype: float64

In [135]:
compare('Adult/Child',data)

Adult/Child
Adult    24.965454
Child    32.280702
Name: Survival, dtype: float64

In [19]:
trainingData = data[["Age","Crew/Pass","Class","Adult/Child","Gender","Survival"]]
trainingData.head()

,Age,Crew/Pass,Class,Adult/Child,Gender,Survival
0,27.0,Passenger,3rd,Adult,Male,1
1,42.0,Passenger,3rd,Adult,Male,0
2,39.0,Passenger,3rd,Adult,Female,1
3,16.0,Passenger,3rd,Child,Male,0
4,13.0,Passenger,3rd,Child,Male,0


In [20]:
def compare(group, data):
    return data.groupby([group])['Survival'].sum()*100/data.groupby([group])['Survival'].count()

In [21]:
def catToNum(series):
    series = series.astype('category')
    return series.cat.codes

catData = trainingData[["Crew/Pass","Class","Adult/Child","Gender"]].apply(catToNum)
trainingData[["Crew/Pass","Class","Adult/Child","Gender"]] = catData

c:\users\manishpeshwani\appdata\local\programs\python\python36-32\lib\site-packages\pandas\core\frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [22]:
trainingData.head()

,Age,Crew/Pass,Class,Adult/Child,Gender,Survival
0,27.0,1,2,0,1,1
1,42.0,1,2,0,1,0
2,39.0,1,2,0,0,1
3,16.0,1,2,1,1,0
4,13.0,1,2,1,1,0


In [23]:
len(trainingData)
type(trainingData)

pandas.core.frame.DataFrame

In [24]:
trainingData = trainingData.dropna()
len(trainingData)

2426

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
train, test = train_test_split(trainingData, test_size=0.2)

In [27]:
train.head()

,Age,Crew/Pass,Class,Adult/Child,Gender,Survival
110,32.0,1,2,0,0,1
937,45.0,1,1,0,1,0
160,27.0,0,3,0,0,0
299,28.0,0,3,0,1,0
16,31.0,0,3,0,1,0


In [28]:
test.head()

,Age,Crew/Pass,Class,Adult/Child,Gender,Survival
639,37.0,1,2,0,1,0
336,35.0,0,3,0,1,0
1205,30.0,0,3,0,1,0
2302,18.0,0,3,1,1,0
1510,32.0,1,2,0,0,0


In [29]:
len(test)

486

In [30]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf = clf.fit(train[["Age","Crew/Pass","Class","Adult/Child","Gender"]],train[["Survival"]])

In [31]:
clf.feature_importances_

array([ 0.40298332,  0.00419927,  0.19191435,  0.0005363 ,  0.40036676])

In [32]:
from sklearn import tree

In [34]:
with open("titanic.dot","w") as f:
    f=tree.export_graphviz(clf,feature_names=["Age","Crew/Pass","Class","Adult/Child","Gender"],out_file=f)

In [35]:
predictions=clf.predict(test[["Age","Crew/Pass","Class","Adult/Child","Gender"]])

In [36]:
from sklearn.metrics import accuracy_score
accuracy_score(test["Survival"],predictions)

0.81069958847736623

In [47]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000)

In [48]:
def checkAccuracy(clf):
    clf = clf.fit(train[["Age","Crew/Pass","Class","Adult/Child","Gender"]],train[["Survival"]])
    predictions=clf.predict(test[["Age","Crew/Pass","Class","Adult/Child","Gender"]])
    return accuracy_score(test["Survival"],predictions)
                  

In [49]:
checkAccuracy(clf)

c:\users\manishpeshwani\appdata\local\programs\python\python36-32\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.81481481481481477

In [50]:
from xgboost.sklearn import XGBClassifier

ModuleNotFoundError: No module named 'xgboost'